#### READING TEST SCORES

The Programme for International Student Assessment (PISA) is a test given every three years to 15-year-old students from around the world to evaluate their performance in mathematics, reading, and science. This test provides a quantitative way to compare the performance of students from different parts of the world. In this homework assignment, we will predict the reading scores of students from the United States of America on the 2009 PISA exam.

The datasets pisa2009train.csv and pisa2009test.csv contain information about the demographics and schools for American students taking the exam, derived from 2009 PISA Public-Use Data Files distributed by the United States National Center for Education Statistics (NCES). While the datasets are not supposed to contain identifying information about students taking the test, by using the data you are bound by the NCES data use agreement, which prohibits any attempt to determine the identity of any student in the datasets.

Each row in the datasets pisa2009train.csv and pisa2009test.csv represents one student taking the exam.

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import seaborn as sns
sns.set_style('darkgrid')

from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm

import matplotlib.pyplot as plt
%matplotlib inline

#from sklearn import datasets, linear_model, metrics

#import itertools
#import pandas_datareader.data as pdweb
#from pandas_datareader.data import DataReader
#from datetime import datetime
#from io import StringIO

In [4]:
pisaTrain = pd.read_csv('../data/pisa2009train.csv')
pisaTest = pd.read_csv('../data/pisa2009test.csv')
pisaTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3663 entries, 0 to 3662
Data columns (total 24 columns):
grade                    3663 non-null int64
male                     3663 non-null int64
raceeth                  3628 non-null object
preschool                3607 non-null float64
expectBachelors          3601 non-null float64
motherHS                 3566 non-null float64
motherBachelors          3266 non-null float64
motherWork               3570 non-null float64
fatherHS                 3418 non-null float64
fatherBachelors          3094 non-null float64
fatherWork               3430 non-null float64
selfBornUS               3594 non-null float64
motherBornUS             3592 non-null float64
fatherBornUS             3550 non-null float64
englishAtHome            3592 non-null float64
computerForSchoolwork    3598 non-null float64
read30MinsADay           3629 non-null float64
minutesPerWeekEnglish    3477 non-null float64
studentsInEnglish        3414 non-null float64
schoo

In [11]:
#What is the average reading score of males?
pisaTrain[pisaTrain['male']==1].mean()

grade                      10.036859
male                        1.000000
preschool                   0.729246
expectBachelors             0.754615
motherHS                    0.889746
motherBachelors             0.382902
motherWork                  0.729418
fatherHS                    0.865330
fatherBachelors             0.367153
fatherWork                  0.863662
selfBornUS                  0.936854
motherBornUS                0.778747
fatherBornUS                0.770166
englishAtHome               0.868450
computerForSchoolwork       0.894823
read30MinsADay              0.192121
minutesPerWeekEnglish     267.769231
studentsInEnglish          24.318630
schoolHasLibrary            0.973199
publicSchool                0.915598
urban                       0.402244
schoolSize               1380.298716
readingScore              483.532479
dtype: float64

In [12]:
#What is the average reading score of females?
pisaTrain[pisaTrain['male']==0].mean()

grade                      10.145170
male                        0.000000
preschool                   0.715986
expectBachelors             0.818647
motherHS                    0.869863
motherBachelors             0.312150
motherWork                  0.739703
fatherHS                    0.852959
fatherBachelors             0.295320
fatherWork                  0.841980
selfBornUS                  0.925441
motherBornUS                0.766079
fatherBornUS                0.763218
englishAtHome               0.875000
computerForSchoolwork       0.904141
read30MinsADay              0.391892
minutesPerWeekEnglish     264.593329
studentsInEnglish          24.683619
schoolHasLibrary            0.961828
publicSchool                0.953099
urban                       0.366834
schoolSize               1357.814620
readingScore              512.940631
dtype: float64

In [14]:
#Which columns are missing data in at least one row? Two ways to do it:

#1
len(pisaTrain.index)-pisaTrain.count()

#2
pisaTrain.isnull().sum()

grade                      0
male                       0
raceeth                   35
preschool                 56
expectBachelors           62
motherHS                  97
motherBachelors          397
motherWork                93
fatherHS                 245
fatherBachelors          569
fatherWork               233
selfBornUS                69
motherBornUS              71
fatherBornUS             113
englishAtHome             71
computerForSchoolwork     65
read30MinsADay            34
minutesPerWeekEnglish    186
studentsInEnglish        249
schoolHasLibrary         143
publicSchool               0
urban                      0
schoolSize               162
readingScore               0
dtype: int64

In [16]:
# Now impute the data / get rid of rows with missing values
pisaTrain = pisaTrain.dropna()
pisaTest = pisaTest.dropna()
pisaTrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2414 entries, 1 to 3662
Data columns (total 24 columns):
grade                    2414 non-null int64
male                     2414 non-null int64
raceeth                  2414 non-null object
preschool                2414 non-null float64
expectBachelors          2414 non-null float64
motherHS                 2414 non-null float64
motherBachelors          2414 non-null float64
motherWork               2414 non-null float64
fatherHS                 2414 non-null float64
fatherBachelors          2414 non-null float64
fatherWork               2414 non-null float64
selfBornUS               2414 non-null float64
motherBornUS             2414 non-null float64
fatherBornUS             2414 non-null float64
englishAtHome            2414 non-null float64
computerForSchoolwork    2414 non-null float64
read30MinsADay           2414 non-null float64
minutesPerWeekEnglish    2414 non-null float64
studentsInEnglish        2414 non-null float64
schoo

In [17]:
pisaTest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990 entries, 0 to 1569
Data columns (total 24 columns):
grade                    990 non-null int64
male                     990 non-null int64
raceeth                  990 non-null object
preschool                990 non-null float64
expectBachelors          990 non-null float64
motherHS                 990 non-null float64
motherBachelors          990 non-null float64
motherWork               990 non-null float64
fatherHS                 990 non-null float64
fatherBachelors          990 non-null float64
fatherWork               990 non-null float64
selfBornUS               990 non-null float64
motherBornUS             990 non-null float64
fatherBornUS             990 non-null float64
englishAtHome            990 non-null float64
computerForSchoolwork    990 non-null float64
read30MinsADay           990 non-null float64
minutesPerWeekEnglish    990 non-null float64
studentsInEnglish        990 non-null float64
schoolHasLibrary         

In [23]:
# Now use get_dummies on raceeth. Use White as the reference level as it is the most frequently occurring level in the data.

dummy_df = pd.get_dummies(pisaTrain['raceeth'], prefix='race', prefix_sep ='_')
# del dummy_df['race_White']
dummy_df = dummy_df.drop(['race_White'], axis = 1)
dummy_df[:5]

,race_American Indian/Alaska Native,race_Asian,race_Black,race_Hispanic,race_More than one race,race_Native Hawaiian/Other Pacific Islander
1,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,1.0,0.0


In [25]:
pisaTrain = pisaTrain.join(dummy_df)
pisaTrain.drop(['raceeth'], axis = 1 , inplace= True, errors= 'ignore')
pisaTrain[:5]

,grade,male,preschool,expectBachelors,motherHS,motherBachelors,motherWork,fatherHS,fatherBachelors,fatherWork,...,publicSchool,urban,schoolSize,readingScore,race_American Indian/Alaska Native,race_Asian,race_Black,race_Hispanic,race_More than one race,race_Native Hawaiian/Other Pacific Islander
1,11,1,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,1,0,1173.0,575.01,0.0,0.0,0.0,0.0,0.0,0.0
3,10,0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,1,1,2640.0,458.11,0.0,0.0,1.0,0.0,0.0,0.0
4,10,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1,1,1095.0,613.89,0.0,0.0,0.0,1.0,0.0,0.0
7,10,0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1,0,1913.0,439.36,0.0,0.0,0.0,0.0,0.0,0.0
9,10,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,...,1,0,899.0,465.90,0.0,0.0,0.0,0.0,1.0,0.0


In [30]:
#Python doesn't like the name: race_Native Hawaiian/Other Pacific Islander
pisaTrain.columns.values

array(['grade', 'male', 'preschool', 'expectBachelors', 'motherHS',
       'motherBachelors', 'motherWork', 'fatherHS', 'fatherBachelors',
       'fatherWork', 'selfBornUS', 'motherBornUS', 'fatherBornUS',
       'englishAtHome', 'computerForSchoolwork', 'read30MinsADay',
       'minutesPerWeekEnglish', 'studentsInEnglish', 'schoolHasLibrary',
       'publicSchool', 'urban', 'schoolSize', 'readingScore',
       'race_American Indian/Alaska Native', 'race_Asian', 'race_Black',
       'race_Hispanic', 'race_More than one race',
       'race_Native Hawaiian/Other Pacific Islander'], dtype=object)

In [33]:
pisaTrain.columns = ['grade', 'male', 'preschool', 'expectBachelors', 'motherHS',
       'motherBachelors', 'motherWork', 'fatherHS', 'fatherBachelors',
       'fatherWork', 'selfBornUS', 'motherBornUS', 'fatherBornUS',
       'englishAtHome', 'computerForSchoolwork', 'read30MinsADay',
       'minutesPerWeekEnglish', 'studentsInEnglish', 'schoolHasLibrary',
       'publicSchool', 'urban', 'schoolSize', 'readingScore',
       'race_American_Indian_or_Alaska_Native', 'race_Asian', 'race_Black',
       'race_Hispanic', 'race_More_than_one',
       'race_Native_Hawaiian_or_Other_Pacific_Islander']

In [34]:
all_columns = "+".join(pisaTrain.columns - ["readingScore"])
my_formula = 'readingScore ~' + all_columns
LinReg = smf.ols(formula=my_formula, data = pisaTrain).fit()
LinReg.summary()

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()
  if __name__ == '__main__':


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           readingScore   R-squared:                       0.325
Model:                            OLS   Adj. R-squared:                  0.317
Method:                 Least Squares   F-statistic:                     41.04
Date:                Tue, 19 Jul 2016   Prob (F-statistic):          1.72e-180
Time:                        11:33:18   Log-Likelihood:                -13795.
No. Observations:                2414   AIC:                         2.765e+04
Df Residuals:                    2385   BIC:                         2.781e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
==================================================================================================================
                                                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------------------------
Intercept                                        143.7663     33.841      4.248      0.000        77.405   210.128
computerForSchoolwork                             22.5002      5.703      3.946      0.000        11.318    33.683
englishAtHome                                      8.0357      6.859      1.171      0.242        -5.415    21.487
expectBachelors                                   55.2671      4.294     12.871      0.000        46.847    63.687
fatherBachelors                                   16.9298      3.995      4.237      0.000         9.095    24.764
fatherBornUS                                       4.3070      6.264      0.688      0.492        -7.976    16.590
fatherHS                                           4.0182      5.579      0.720      0.471        -6.923    14.959
fatherWork                                         5.8428      4.396      1.329      0.184        -2.778    14.463
grade                                             29.5427      2.937     10.057      0.000        23.783    35.303
male                                             -14.5217      3.156     -4.601      0.000       -20.710    -8.333
minutesPerWeekEnglish                              0.0128      0.011      1.194      0.233        -0.008     0.034
motherBachelors                                   12.6381      3.861      3.273      0.001         5.066    20.210
motherBornUS                                      -8.7982      6.588     -1.336      0.182       -21.716     4.120
motherHS                                           6.0588      6.091      0.995      0.320        -5.886    18.004
motherWork                                        -2.8091      3.522     -0.798      0.425        -9.715     4.097
preschool                                         -4.4637      3.486     -1.280      0.201       -11.300     2.372
publicSchool                                     -16.8575      6.726     -2.506      0.012       -30.046    -3.669
race_American_Indian_or_Alaska_Native            -67.2773     16.787     -4.008      0.000      -100.196   -34.359
race_Asian                                        -4.1103      9.220     -0.446      0.656       -22.191    13.970
race_Black                                       -67.0123      5.461    -12.271      0.000       -77.721   -56.304
race_Hispanic                                    -38.9755      5.178     -7.528      0.000       -49.129   -28.822
race_More_than_one                               -16.9225      8.496     -1.992      0.047       -33.583    -0.262
race_Native_Hawaiian_or_Other_Pacific_Islander    -5.1016     17.006     -0.300      0.764       -38.449    28.246
read30MinsADay                                    34.8719      3.408     10.231      0.000   

In [35]:
# What is root mean squared error of this model?
SSE = sum(LinReg.resid**2)
RMSE = np.sqrt(SSE/len(pisaTrain))
print('RMSE = ',RMSE)

RMSE =  73.365551433


In [39]:
# Now predict the difference in reading scores of two similar students one in grade 9 and one in grade 11.

LinReg.params['grade']*11 - LinReg.params['grade']*9

59.08541418353866

In [40]:
LinReg.tvalues

Intercept                                          4.248260
computerForSchoolwork                              3.945636
englishAtHome                                      1.171469
expectBachelors                                   12.871088
fatherBachelors                                    4.237468
fatherBornUS                                       0.687593
fatherHS                                           0.720204
fatherWork                                         1.329124
grade                                             10.057438
male                                              -4.601392
minutesPerWeekEnglish                              1.193878
motherBachelors                                    3.272875
motherBornUS                                      -1.335558
motherHS                                           0.994640
motherWork                                        -0.797626
preschool                                         -1.280436
publicSchool                            

In [46]:
LinReg.t_test([1,1,1,1,1,22,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5,5,5,5,6,6,6,6])

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
c0          -321.3024    209.855     -1.531      0.126      -732.820    90.215

In [45]:
LinReg.f_test([1,1,1,1,1,22,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5,5,5,5,6,6,6,6])

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[ 2.34416245]]), p=0.12588575506292887, df_denom=2385, df_num=1>

In [55]:
# Now predict with pisaTest data. (out of sample prediction)
# Note that we probably need to add dummy variables and clean up column names again...

dummy_df = pd.get_dummies(pisaTest['raceeth'], prefix='race', prefix_sep ='_')
dummy_df = dummy_df.drop(['race_White'], axis = 1)

pisaTest = pisaTest.join(dummy_df)
pisaTest = pisaTest.drop(['raceeth'], axis = 1)

pisaTest.columns = ['grade', 'male', 'preschool', 'expectBachelors', 'motherHS',
       'motherBachelors', 'motherWork', 'fatherHS', 'fatherBachelors',
       'fatherWork', 'selfBornUS', 'motherBornUS', 'fatherBornUS',
       'englishAtHome', 'computerForSchoolwork', 'read30MinsADay',
       'minutesPerWeekEnglish', 'studentsInEnglish', 'schoolHasLibrary',
       'publicSchool', 'urban', 'schoolSize', 'readingScore',
       'race_American_Indian_or_Alaska_Native', 'race_Asian', 'race_Black',
       'race_Hispanic', 'race_More_than_one',
       'race_Native_Hawaiian_or_Other_Pacific_Islander']

In [62]:
predTest = LinReg.predict(pisaTest)
print('min=',predTest.min(),'max=',predTest.max(),'range=',predTest.max()-predTest.min())

min= 353.223123114 max= 637.691434909 range= 284.468311795


In [64]:
# What is the sum of squared errors (SSE) on the testing set?

SSE = sum((predTest-pisaTest['readingScore'])**2)
print('SSE =',SSE)
RMSE = np.sqrt(SSE/len(pisaTest))
print('RMSE = ',RMSE)

SSE = 5762082.37114
RMSE =  76.2907938311


In [68]:
# What is the predicted test score used in the baseline model? Compute this value using the training set and not the test set.

predTrain = LinReg.predict(pisaTrain)
print('mean of predicted readingScores on pisaTrain: ',predTrain.mean())

print('mean of pisaTrain readingScore: ',pisaTrain['readingScore'].mean())

mean of predicted readingScores on pisaTrain:  517.962887324
mean of pisaTrain readingScore:  517.9628873239429


In [72]:
# What is the sum of squared errors of the baseline model on the testing set?
# We call the sum of squared errors for the baseline model the total sum of squares (SST).

SST = sum((predTrain.mean()-pisaTest['readingScore'])**2)
print('SST =',SST)

SST = 7802354.07761


## What is the test-set R-squared?

In [76]:
# The test-set R^2 is defined as 1-SSE/SST, where SSE is the sum of squared errors of the model on the test set
# and SST is the sum of squared errors of the baseline model.

SSE = sum((predTest-pisaTest['readingScore'])**2)
print('SSE =',SSE)

SST = sum((predTrain.mean()-pisaTest['readingScore'])**2)
print('SST =',SST)

print('test-set R^2: ',1-SSE/SST)

SSE = 5762082.37114
SST = 7802354.07761
test-set R^2:  0.261494375438
